# ISL Dataprep
## Run DWPose on downsampled and trimmed videos

## Set up data

Data Source: Lambda filesystem
  - bucket name: 921c05b3-8e02-4c0b-9623-d1de39939163
  - region: us-east-3


AWS creds needed:
  - AWS_ACCESS_KEY_ID=9P7Q2ADQR5WPGY83629B
  - AWS_SECRET_ACCESS_KEY=*********
  - AWS_REGION=us-east-3
  - S3_ENDPOINT_URL="https://files.us-east-3.lambda.ai"

Change the bucket and credentials when using a different instance. These are given as pattern reference.

In [ ]:
!pip install awscli boto3

In [2]:
! aws --version

aws-cli/1.42.18 Python/3.12.11 Linux/6.1.123+ botocore/1.40.18


In [ ]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = '9P7Q2ADQR5WPGY83629B'
os.environ['AWS_SECRET_ACCESS_KEY'] = '****'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-3'

In [ ]:
! aws configure

In [4]:
!echo "[default]" > ~/.aws/config
!echo "region =us-east-3 " >> ~/.aws/config
!echo "endpoint-url =https://files.us-east-3.lambda.ai " >> ~/.aws/config
!echo "" >> ~/.aws/config


!echo "request_checksum_calculation =when_required " >> ~/.aws/config
!echo "response_checksum_validation =when_required " >> ~/.aws/config
!echo "" >> ~/.aws/config


In [5]:
cat ~/.aws/config

[default]
region =us-east-3 
endpoint-url =https://files.us-east-3.lambda.ai 

request_checksum_calculation =when_required 
response_checksum_validation =when_required 



In [6]:
!aws s3 ls s3://921c05b3-8e02-4c0b-9623-d1de39939163/ --endpoint-url https://files.us-east-3.lambda.ai

                           PRE ENV/
                           PRE code/
                           PRE isl_dict_videos/
                           PRE isl_gospel_videos/
                           PRE logs/
                           PRE models/
                           PRE outputs/


In [ ]:
!aws s3 cp s3://921c05b3-8e02-4c0b-9623-d1de39939163/isl_gospel_videos/ /content/isl_gospel_videos --recursive --endpoint-url https://files.us-east-3.lambda.ai

In [9]:
! aws s3 cp s3://921c05b3-8e02-4c0b-9623-d1de39939163/models/ /content/models --recursive --endpoint-url https://files.us-east-3.lambda.ai

download: s3://921c05b3-8e02-4c0b-9623-d1de39939163/models/dw-ll_ucoco_384.onnx to models/dw-ll_ucoco_384.onnx
download: s3://921c05b3-8e02-4c0b-9623-d1de39939163/models/yolox_l.onnx to models/yolox_l.onnx


In [10]:
! df -h /

Filesystem      Size  Used Avail Use% Mounted on
overlay         236G   64G  172G  28% /


In [11]:
ls /content/

isl_gospel_videos/  models/  sample_data/


## Set up code

In [12]:
! mkdir /content/logs

In [13]:
! mkdir /content/dwpose_output

In [ ]:
! git clone https://github.com/kavitharaju/sign-bibles-dataset.git

In [15]:
cd /content/sign-bibles-dataset/dataprep/isl/

/content/sign-bibles-dataset/dataprep/isl


In [ ]:
! ls /content/sign-bibles-dataset/dataprep/isl

In [17]:
!git checkout -b separate-pipelines-4-GPU-CPU

Switched to a new branch 'separate-pipelines-4-GPU-CPU'


In [ ]:
!git pull origin separate-pipelines-4-GPU-CPU

In [ ]:
!apt-get update && apt-get install -y parallel

In [20]:
!parallel --version

GNU parallel 20210822
Copyright (C) 2007-2021 Ole Tange, http://ole.tange.dk and Free Software
Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
GNU parallel comes with no warranty.

Web site: https://www.gnu.org/software/parallel

When using programs that use GNU Parallel to process data for publication
please cite as described in the manpage.


In [ ]:
! pip install -r requirements-dwpose-on_GPU.txt

## Run data processing

In [22]:
!ls /content/isl_gospel_videos/*.mp4 | xargs -n 1 basename -s .mp4 > full_input_list.txt

In [23]:
! sed -n '1,50p' full_input_list.txt > input_list.txt

In [ ]:
!time(bash run_parallel_gpu.sh)

In [ ]:
cat /content/logs/success.log

## Upload outputs

In [27]:
pwd

'/content/sign-bibles-dataset/dataprep/isl'

In [ ]:
! ls /content/dwpose_output | wc

In [ ]:
! aws s3 cp /content/dwpose_output/ s3://921c05b3-8e02-4c0b-9623-d1de39939163/outputs/ --recursive --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.npz"

In [ ]:
! aws s3 ls s3://921c05b3-8e02-4c0b-9623-d1de39939163/outputs/ --endpoint-url=https://files.us-east-3.lambda.ai | wc

   1138    4552   58080


In [ ]:
! rm  /content/dwpose_output/*